In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
from konlpy.tag import Mecab
#from eunjeon import Mecab
from hanspell import spell_checker

In [3]:
E_list = [
    
    'EF', 'EP', 'VCP', 'VCN', 'XSA', 'XSV', 'NP', 'JX', 'JKB', #이는 어말 처리를 위한 것들이다. 어말 부분에 추가할 것이 있다면 여기에
                                                              
]

con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

In [28]:
mec = Mecab()

def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

#Create list words & tags
def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

#remember location of space and store in list(input == ' ')
def rememberSpace(lis, input):
    rlis = []
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i) #remember index of space
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i      
    return rlis

def convertSpace(lis_space,lis_lis):
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    #print(rlis)  
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1
#spacing으로 단위를 바꿨음        
def union_t(lis_lis):
    
    for i in range(len(lis_lis)-1):
        if lis_lis[i+1] != ' ' and lis_lis[i+1] !='SF' and lis_lis[i] != ' ' and lis_lis[i] !='SF':
            lis_lis[i] = lis_lis[i]+'/'
        if lis_lis[i+1]=='SF':
            lis_lis[i+1] = ' '+lis_lis[i+1]
            
def union_w(lis_w, lis_tag):
    
    for i in range(len(lis_w)-1):
        if lis_tag[i+1]==' SF':
            lis_w[i+1] = ' '+lis_w[i+1]
            
def prepro(input):
    lis_w, lis_t = to2lists(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t(lis_t)
    union_w(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    for i in range(len(data_w)-1):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1
        
        #element = element[:-1]
        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
    
    return data_w, data_t, lis_ind

def pre_target(input, lis, target):#tags , tag list, target
    result = input
    if target in input:
        res = input.split('/')
        for j in range(len(res)):
            if target in res[j] and '+' in res[j]:
                loc = res[j].index('+')
                wd = res[j][:loc]
                print(wd)
                if wd not in lis: #
                    res[j] = re.sub(wd, 'A', res[j])
        result = '/'.join(res)
    return result

def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        for j in range(len(i)):
            gre = li2str(i[j])
            bullet.append(gre)
        result.append(bullet)
    return result

def isException(input): #special case?
    if 'special' in input[1]:
        return 1
    else:
        return 0

In [29]:
inin = 'XSA/CC+EF'
rere = pre_target(inin, E_list, 'EF')
print(rere)

CC
XSA/A+EF


In [13]:
##EF(종결어미): 형태소 분류 시 종결어미가 단독으로 분석 되는 경우
EF_ONLY = [
    
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']], #ex)'습니다'-> '다'
    [['ㄷ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㄹ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㄴ','ㅏ','ㅇ','ㅛ'],['ㄴㅏ']],
    [['ㄷ','ㅐ','ㅇ','ㅛ'],['ㄷ','ㅐ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄴ','ㅔ','ㅇ','ㅛ'],['ㄷ','ㅏ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅈ','ㅛ'],['ㅈ','ㅣ']],
    [['ㅇ','ㅏ','ㅇ','ㅛ'],['ㅇ','ㅏ']],
    [['ㅇ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅓ']],
    [['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ'],['specialx']],
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ']],
    #[['ㅎ','ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅎ','ㅇ','ㅏ']],
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅓ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㄷㅏ']],
    [['ㅇ','ㅛ'],['']],
    
]

#EP(선어말어미)+EF(종결어미)로 형태소가 분석될 경우
EP_EF = [
    
    [['ㅅ','ㅔ','ㅇ','ㅛ'],['specialx']]
    #-시오 family
    ,[['ㅡ','ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㅓ']]
    ,[['ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㄹ', 'ㅏ']],
    [['ㅅ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅣ']],
    [['ㅅ','ㅣ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ'],['ㄹ','ㄹ','ㅐ']],
]

#VCP+EF
VCP_EF = [
    
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['-special']],
    [['ㅇ','ㅣ','ㄴ','ㄱ','ㅏ','ㅇ','ㅛ'],['-special']],
    [['ㅈ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅣ','ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄱ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ']],
    [['ㄹ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㄷ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    
]

VCN_EF = [
    
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅇ','ㅑ']],
    
]

#XSA+EF
XSA_EF = [
    [['ㅎ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅎ','ㅏ','ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
]

#XSV+EF
XSV_EF = [
    [['ㅎ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅎ','ㅐ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['']],
    [['ㅐ','ㅇ','ㅛ'],['ㅐ']],
]

#alpha+EF: 위의 경우를 제외한 나머지 경우를 의미한다. 
A_EF = [
    [['ㄱ','ㅓ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㄱ','ㅖ','ㅅ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅆ','ㅇ','ㅓ']],
    [['ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅏ']],
    [['ㅇ','ㅛ'],['']],
    [['ㅈ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄷ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']]
    
    
]

##높임표현이 어말어미가 아닌 경우##
# - 기존의 변환 방식이었다. 이제는 쓰지는 않지만, 코드의 오류가 발생할 가능성이 있기에 지우지 않았다. 따로 참고할 필요 없음.
EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
    #,[['ㅅ','ㅔ'],['ㅝ'],['ㅘ']]
]

NP = [
    
    [['ㅈ','ㅓ','ㄴ'],['special-']],
    [['ㅈ','ㅓ'],['ㄴ','ㅏ']]
    
]

# 보조사
JX = [
    [['ㅇ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
]

JKB_JX = [
    [['ㅇ','ㅛ'],['specialx']]
]



VX = [
    
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']]
    
]

VV = [
    
    [['ㅇ','ㅗ'],['ㅇ','ㅗ'],['ㅇ','ㅘ']],
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']],
    [['ㅎ','ㅏ'],['ㅎ','ㅏ'],['ㅎ','ㅐ']]
    
]

XR = [
    [['ㅈ','ㅚ','ㅅ','ㅗ','ㅇ'],['ㅁ','ㅣ','ㅇ','ㅏ','ㄴ'] ]
]

NNG = [
    [['ㅈ','ㅏ','ㅇ','ㅛ'],['ㅈ','ㅏ']],
    #[['ㄱ','ㅏ','ㅁ','ㅅ','ㅏ'],['ㅈ','ㅏ']],
]

NNP =[
    [[],[]]
]

EC = [
    
    [['ㅇ','ㅛ'],['ㄷ','ㅏ']]
    
]

EXC_deal_1 = [
    
    ['ㅅㅔㅇㅛ', ['ㅏ','ㅣ'],['ㅜ','ㅓ'],['ㄹㅡ','ㄹㄹㅓ'],['ㅡ','ㅓ'],['ㅣ','ㅓ']],
]

EXC_deal_2 = [
    
    ['ㅈㅓㄴ','ㅈㅓㄴ','ㄴㅏㄴ'],
]

EXC_deal_3 = [
    
    ['ㄴㅏㅇㅛ',[['ㅆ', 'ㅇㅓ']],'아'],
    ['ㅅㅡㅂㄴㅣㄷㅏ',[['ㅆ','ㄷㅏ'],['ㅎ','ㄷㅏ'],['ㅄ','다'],['ㅂ','ㄷㅏ'],['ㅌ','ㄷㅏ'],['ㅍ','ㄷㅏ'],['ㄵ','ㄷㅏ']],'ㄴㅡㄴㄷㅏ'],
    ['ㅇㅣㅂㄴㅣㄲㅏ',[['ㄹ','ㅇㅣㅇㅑ'],['ㅂ','ㅇㅣㅇㅑ'],['ㅅ','ㅇㅣㅇㅑ'],['ㄷ','ㅇㅣㅇㅑ'],['ㅁ','ㅇㅣㅇㅑ'],['ㄴ','ㅇㅣㅇㅑ'],['ㄱ','ㅇㅣㅇㅑ'],['ㅄ','ㅇㅣㅇㅑ'],['ㅇ','ㅇㅣㅇㅑ'],['ㅈㅗ','ㅇㅕ'],['ㅂㅗ','ㅇㅕ']],'ㅇㅑ'],
    ['ㅇㅣㅂㄴㅣㄷㅏ',[['ㄹ','ㅇㅣㄷㅏ'],['ㅂ','ㅇㅣㄷㅏ'],['ㅅ','ㅇㅣㄷㅏ'],['ㄷ','ㅇㅣㄷㅏ'],['ㅁ','ㅇㅣㄷㅏ'],['ㄴ','ㅇㅣㄷㅏ'],['ㄱ','ㅇㅣㄷㅏ'],['ㅄ','ㅇㅣㄷㅏ'],['ㅇ','ㅇㅣㄷㅏ'],['ㅈㅗ','ㅇㅣㄴㄷㅏ'],['ㅂㅗ','ㅇㅣㄴㄷㅏ']],'ㄷㅏ'],
    ['ㅇㅣㄴㄱㅏㅇㅛ',[['ㄱ','ㅇㅣㅇㅑ'],['ㄴ','ㅇㅣㅇㅑ'],['ㄷ','ㅇㅣㅇㅑ'],['ㄹ','ㅇㅣㅇㅑ'],['ㅁ','ㅇㅣㅇㅑ'],['ㅂ','ㅇㅣㅇㅑ'],['ㅇ','ㅇㅣㅇㅑ'],['ㅄ','ㅇㅣㅇㅑ'],['ㅀ','ㅇㅣㅇㅑ']],'ㅇㅑ'],
    ['ㄷㅏㅂㄴㅣㄷㅏ',[['','']],'ㄷㅏㄴㄷㅏ'],
    ['ㄹㅏㅂㄴㅣㄷㅏ',[['','']],'ㄹㅏㄴㄷㅏ'],
    
]

EXC_deal_4 = [

    ['ㅇㅛ','',[['ㅇㅔ','ㅇㅣ','ㄷㅏ'],''],''],
    ['ㅅㅔㅇㅛ', 'ㄷㅏ', [['ㅏ','ㅏ','ㅣ'],['ㅜㄹㅡ','ㅜㄹ','ㄹㅓ'],['ㅜ','ㅜ','ㅓ'],['ㅡ','ㅡ','ㅓ'],['ㅐ','ㅐ',''],['ㅣ','ㅣ','ㅓ'],'ㅇㅓ'],'ㅇㅑ'],
    ['ㅇㅡㅅㅔㅇㅛ','ㄷㅏ',[['ㄴㅗㅎ','ㄴㅗㅏ',''],''],'ㅇㅏ'],

]

EXC_deal_l = [
    
    ['ㅅㅡㅂㄴㅣㄷㅏ', [['ㅆ','ㄷㅏ'],['ㅎ','ㄷㅏ'],['ㅄ','다'],['ㅂ','ㄷㅏ'],['ㅌ','ㄷㅏ'],['ㅍ','ㄷㅏ'],['ㄵ','ㄷㅏ']],'ㄷㅏ'],
    
]


EXC_tags = [
    
    'EP',
    'EF'
    
]

UNKNOWN = [
    [['ㄷ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅡ','ㄴ','ㄷ','ㅏ']],
]

####End of high -> low Dictionary####


#### low -> high Dictionary####
## 사전의 순서가 중요하다. 순서를 임의로 바꾸지 말 것
## EF solo. EF 앞에 형태소가 결합형이 아닌 홀로 분석 될 경우 ex) /EP, /EF

EF_ONLY_4S = [
    # -다면서 /EF
    [['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ'],['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ','ㅇ','ㅛ']],
    # -고 /EF
    [['ㄹ','ㅏ','ㄱ','ㅗ'],['ㄹ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']], ##-라고-> 라고요
    [['ㄷ','ㅓ','ㄹ','ㅏ','ㄱ','ㅗ'],['ㄷ','ㅓ','ㄹ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ','ㄱ','ㅗ'],['ㄷ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    #-어서다 /EF
    [['ㅇ','ㅓ','ㅅ','ㅓ','ㄷ','ㅏ'],['ㅇ','ㅓ','ㅅ','ㅓ','ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㄷ','ㅏ'],['-special']],
    [['ㅇ','ㅓ','ㄹ','ㅏ'],['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ']], #add jh_1115
    [['ㅇ','ㅓ'],['ㅇ','ㅓ','ㅇ','ㅛ']],
    #'-라' /EF
    [['ㄷ','ㅓ','ㄹ','ㅏ'],['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅇ','ㅏ','ㄹ','ㅏ'],['ㅇ','ㅏ','ㅇ','ㅛ']],
    [['ㄹ','ㅣ','ㄹ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']], ##'-리라'는 아직 해결하지 못하였다. 
    [['ㄹ','ㅏ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    
    
    # -데 /EF
    [['ㅇ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㅇ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ'],['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    # -걸 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ'],['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    
    [['ㅈ','ㅏ','ㄶ','ㅇ','ㅏ'],['ㅈ','ㅏ','ㄶ','ㅇ','ㅏ','ㅇ','ㅛ']], ## -잖아 -> -잖아요
    
    #-오 /EF
    [['ㅇ','ㅗ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    #-니 /EF
    [['ㄴ','ㅣ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    #-자 /EF
    [['ㅈ','ㅏ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    #-래 /EF
    [['ㅇ''ㅡ','ㄹ','ㄹ','ㅐ'],['ㅇ''ㅡ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    
    
    # -거든 /EF
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ']],
    
    #-마 /EF
    [['ㅁ','ㅏ'],['ㅁ','ㅏ','ㅇ','ㅛ']],
    
    #-아 /EF
    [['ㅇ','ㅏ'],['ㅇ','ㅏ','ㅇ','ㅛ']],
    #-지/EF
    [['ㅈ','ㅣ'],['ㅈ','ㅛ']],
    
    #-나 /EF
    [['ㄴ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],

    #[['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['-special']],
    #-는가 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    #-던가 /EF
    [['ㄷ','ㅓ','ㄴ','ㄱ','ㅏ'],['ㄷ','ㅓ','ㄴ','ㄱ','ㅏ','ㅇ','ㅛ']],
    #-을까 /EF
    [['ㅇ''ㅡ','ㄹ','ㄲ','ㅏ'],['ㅇ''ㅡ','ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
    
    # -ㅏ /EF
    [['ㅏ'],['ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    # -ㅔ /EF
    [['ㅔ'],['ㅔ','ㅇ','ㅛ']],
    
    
    
    
]
## EF 결합형
## EF 앞에 형태소가 결합형인 경우 앞의 형태소도 고려해서 처리해야 한다. ex) /EP, /EF
EF_ONLY_4C = [
    
    [['ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅓ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅓ','ㄸ','ㅐ'],['ㅇ','ㅓ','ㄸ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    
    # -걸 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ'],['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    # -니 /EF
    [['ㄴ','ㅣ'],['ㅇ','ㅓ','ㅇ','ㅛ']],
    # -거든 /EF
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ']],
    [['ㅈ','ㅣ'],['ㅈ','ㅛ']],
    #[['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['-special']],
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    [['ㄴ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    
]

EP_EF_4 =[
    #[['ㄴ','ㅔ'],['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
    [['ㄴ','ㅔ'],['ㄴ','ㅔ','ㅇ','ㅛ']],
]

#/VCP+EF
VCP_EF_4 = [
    #[['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ'],['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']], ##bug check
    [['ㄱ','ㅓ','ㄹ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅇ','ㅑ'],['ㅇ','ㅔ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['ㅇ','ㅔ','ㅇ','ㅛ']], #add jh_1116
    [['ㄷ','ㅔ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ'],['-special']]
    
]

#/VA+EF added code_JH
VA_EF_4 = [
    [['ㅇ','ㅓ','ㄸ','ㅐ'],['ㅇ','ㅓ','ㄸ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    [['ㅕ'],['ㅕ','ㅇ','ㅛ']],
]

## /VCP, /EF
EF_AFTER_VCP_4 = [
    [['ㅇ','ㅑ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    #-오 /EF
    [['ㅇ','ㅗ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['ㅇ','ㅔ','ㅇ','ㅛ']], #add jh_1116
    [['ㄴ','ㅔ'],['ㄴ','ㅔ','ㅇ','ㅛ']],
    
]
#/XSA, /EF
EF_AFTER_XSA_4 = [
    
    [['ㄷ','ㅏ'],['-special']],
    
    
]

VV_EF_4 = [
    [['ㄱ','ㅓ','ㄹ'],['ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ','ㄹ','ㅏ'],['ㅎ','ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ','ㄹ','ㅏ'],['ㅂ','ㅗ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ'],['ㅂ','ㅘ','ㅇ','ㅛ']],
    [['ㅎ','ㅏ','ㄹ','ㄹ','ㅐ'],['ㅎ','ㅏ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ'],['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']], #add jh_1115
    [['ㄷ','ㅙ'],['ㄷ','ㅙ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    [['ㄷ','ㅡ','ㄹ','ㄹ','ㅕ'],['ㄷ','ㅡ','ㄹ','ㄹ','ㅕ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ'],['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅘ'],['ㅇ','ㅘ','ㅇ','ㅛ']], 
    [['ㅊ','ㅝ'],['ㅊ','ㅝ','ㅇ','ㅛ']],
    [['ㅔ'],['ㅔ','ㅇ','ㅛ']],
    [['ㅏ'],['ㅏ','ㅇ','ㅛ']], 
    [['ㅕ'],['ㅕ','ㅇ','ㅛ']],
    
]

##added code_JH
#/XSV+EF
XSV_EF_4 = [
    [['ㄷ','ㅏ'],['special']],
    [['ㅎ','ㅐ','ㄹ','ㅏ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㄲ','ㅏ'],['ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
    [['ㅏ','ㄴ','ㄷ','ㅏ'],['ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    
]
#/XSA+EF
XSA_EF_4 = [
    [['ㄷ','ㅏ'],['special']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅎ','ㅏ','ㄴ','ㄷ','ㅔ'],['ㅎ','ㅏ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    
]
#/VX+EF
VX_EF_4 = [
    [['ㄷ','ㅏ'],['special']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㅏ'],['ㅇ','ㅛ']],
    [['ㅂ','ㅘ','ㄹ','ㅏ'],['ㅂ','ㅗ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ'],['ㅂ','ㅘ','ㅇ','ㅛ']],
    [['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ'],['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']], #add jh_1115
    [['ㅁ','ㅏ'],['ㅁ','ㅏ','ㅇ','ㅛ']],
    [['ㄹ','ㄱ','ㅔ'],['ㄹ','ㄱ','ㅔ','ㅇ','ㅛ']],
    [['ㅝ'],['ㅝ','ㅇ','ㅛ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    
    
]
#/NNB+JKO
NNB_JKO_4 = [
    [['ㄱ','ㅓ','ㄹ'],['specialx']],
]
#/EC
EC_4 = [
    [['ㄷ','ㅔ'],['specialx']],
    #[['ㄷ','ㅔ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄱ','ㅔ'],['specialx']],
    [['ㄴ','ㅔ'],['specialx']],
    #[['ㄱ','ㅔ'],['ㄱ','ㅔ','ㅇ','ㅛ']],
]

#/VV
VV_4 = [
    
    #[['ㄷ','ㅚ'],['ㄷ','ㅚ','ㅇ','ㅛ']],
]
### Exception case dictionary ###
EXC_4_deal_1 = [
    
     ['ㄷㅏ',['ㄴ','ㅂㄴㅣㄷㅏ'],'ㅅㅡㅂㄴㅣㄷㅏ'],
]

EXC_4_deal_2 = [
    
    ['ㄴㅣ',['ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_3 = [
    
    ['ㄴㅡㄴㄱㅏ',['ㅗ','ㅗ','ㅏㅇㅛ'],['ㅜ','ㅜ','ㅓㅇㅛ'],['ㅏ','ㅏ','ㅣㅇㅛ'],'ㅇㅓㅇㅛ'],
    ['ㄷㅏ',['ㅆ','ㅆ','ㅅㅡㅂㄴㅣㄷㅏ'],['ㄴ','ㅂ','ㄴㅣㄷㅏ'],['ㄹ','ㅂ','ㄴㅣㄷㅏ'],['ㅂ','ㅂ','ㅅㅡㅂㄴㅣㄷㅏ'],'ㅂㄴㅣㄷㅏ'],
    ['ㄴㅣ',['ㅆ','ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_4 = [
    
    ['ㄱㅔ','ㄱㅔㅇㅛ',[['ㄵ','ㄵ','ㅇㅡㅅㅔㅇㅛ'],['ㄱ','ㄱ','ㅇㅡㅅㅔㅇㅛ'],['ㄷ','ㄷ','ㅇㅡㅅㅔㅇㅛ'],['ㅜㅂ','ㅜ','ㅇㅡㅅㅔㅇㅛ'],['ㅂ','ㅂ','ㅇㅡㅅㅔㅇㅛ'],'ㅅㅔㅇㅛ'],'ㄱㅔ'],
    ['ㄷㅔ','ㄷㅔㅇㅛ',[['','',''],''],'ㄷㅔ'],
    ['ㄴㅔ','ㄴㅔㅇㅛ',[['','',''],''],'ㄴㅔ'],
    ['ㄱㅓㄹ','ㄱㅓㄹㅇㅛ',[['','',''],''],'ㄱㅓㄹ']
]

#도록은 special

#### End of low -> high Dictionary####

#### Common Dictionary ####
#Yes or No
IC =[
    [['ㄴ','ㅔ'],['ㅇ','ㅡ','ㅇ']],
    [['ㅇ','ㅏ','ㄴ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅏ','ㄴ','ㅣ']],
    [['ㄴ','ㅔ'],['ㄱ','ㅡ','ㄹ','ㅐ']],
]


In [14]:
lil = ['이게무슨','소리야']
jam = Jamodealer(lil)

print(jam.jamo)

['ㅇㅣㄱㅔㅁㅜㅅㅡㄴ', 'ㅅㅗㄹㅣㅇㅑ']


In [15]:
input_1 = '그는 자신이 무엇을 잘못했는지 알지 못한다.'
q,w,e = prepro(input_1)
print(q)
print(w)
print(e)

['그는', '자신이', '무엇을', '잘못했는지', '알지', '못한다', '.']
['NP/JX', 'NNG/JKS', 'NP/JKO', 'MAG/VV+EP/EC', 'VV/EC', 'VX+EF', 'SF']
[[0, 2, 5], [0, 5, 7], [0, 5, 8], [0, 6, 9, 14], [0, 3, 5], [0, 8]]


In [16]:
print(e) #remember the index of Morpheme(형태소의 시작 index를 기억한다. 마지막 index는 형태소 끝을 의미)

[[0, 2, 5], [0, 5, 7], [0, 5, 8], [0, 6, 9, 14], [0, 3, 5], [0, 8]]


In [17]:
print(mec.pos(input_1))
print(Jamodealer(input_1).jamo)

[('그', 'NP'), ('는', 'JX'), ('자신', 'NNG'), ('이', 'JKS'), ('무엇', 'NP'), ('을', 'JKO'), ('잘못', 'MAG'), ('했', 'VV+EP'), ('는지', 'EC'), ('알', 'VV'), ('지', 'EC'), ('못한다', 'VX+EF'), ('.', 'SF')]
['ㄱㅡ', 'ㄴㅡㄴ', ' ', 'ㅈㅏ', 'ㅅㅣㄴ', 'ㅇㅣ', ' ', 'ㅁㅜ', 'ㅇㅓㅅ', 'ㅇㅡㄹ', ' ', 'ㅈㅏㄹ', 'ㅁㅗㅅ', 'ㅎㅐㅆ', 'ㄴㅡㄴ', 'ㅈㅣ', ' ', 'ㅇㅏㄹ', 'ㅈㅣ', ' ', 'ㅁㅗㅅ', 'ㅎㅏㄴ', 'ㄷㅏ', '.']


In [18]:
EF_ONLY_dict = makestrdict(EF_ONLY)
EP_EF_dict = makestrdict(EP_EF)
VCP_EF_dict = makestrdict(VCP_EF)
A_EF_dict = makestrdict(A_EF)
XSA_EF_dict = makestrdict(XSA_EF)
XSV_EF_dict = makestrdict(XSV_EF)
JKB_JX_dict = makestrdict(JKB_JX)
VCN_EF_dict = makestrdict(VCN_EF)
EC_dict = makestrdict(EC)

In [19]:
D_dict = {
    
    'EF':EF_ONLY_dict,
    'EP+EF':EP_EF_dict,
    'VCP+EF':VCP_EF_dict,
    'VCN+EF':VCN_EF_dict,
    'XSA+EF':XSA_EF_dict,
    'XSV+EF':XSV_EF_dict,
    'A+EF':A_EF_dict,
    'JKB+JX':JKB_JX_dict,
    'EC':EC_dict,
    
}

Dict_list = [
    
    'EF', 'EP+EF', 'VCP+EF', 'VCN+EF', 'XSA+EF', 'XSV+EF', 'A+EF', 'JKB+JX', 'EC', 
    
]

In [87]:
class Changer(object):
    #반말로 변경
    def to_low(self, input):
        result = input
        #remember the location of space in sentence
        space_list = rememberSpace(input,' ')
        
        lis_w, lis_t, lis_i = prepro(result)
        space_location = convertSpace(space_list, lis_w)
        print(space_location)
        jam = Jamodealer(lis_w)
        
        lis = []
        for i in range(len(lis_t)):
            pre_i = pre_target(lis_t[i], D_dict, 'EF')
            #print(pre_i)
            t_res = pre_i.split('/')
            #print(t_res)
            element = jam.jamo[i]
            #print(element)    
            for j in range(len(t_res)):
                if t_res[j] in Dict_list: #그 형태소가 사전에 있는지?
                    dic = D_dict[t_res[j]]
                    #print(dic)# 해당하는 사전을 불러온다. 
                    flag = 0
                    
                    for k in range(len(dic)):
                        #해당하는 어미가 사전에 있는지 보고 
                        if dic[k][0] in jam.jamo[i][lis_i[i][j]:lis_i[i][j+1]] and flag==0:
                            
                            flag = 1
                            #special case인가?
                            if isException(dic[k])==0: 
                                res1 = jam.jamo[i][:lis_i[i][j]]
                                print(res1)
                                res2 = jam.jamo[i][lis_i[i][j]:]
                                print(res2) # res2가 바꿔야 할 형태소 
                                res2 = res2.replace(dic[k][0], dic[k][1], 1)
                                element = res1+res2
                                print(element)
                                
                            elif isException(dic[k])==1:
                                flag_01 = 0
                                dic_e = EXC_deal_l
                                for m in range(len(dic_e)):
                                    if dic_e[m][0] in jam.jamo[i][lis_i[i][j]:lis_i[i][j+1]]:
                                        
                                        res1 = jam.jamo[i][:lis_i[i][j]]
                                        print(res1)
                                        res2 = jam.jamo[i][lis_i[i][j]:]
                                        print(res2)
                                        print(dic_e[m][1]) # 변환되어야 할 사전 
                                        for n in dic_e[m][1]:
                                            leng = len(n[0])
                                            print(res1[-leng:])
                                            if res1[-leng:]==n[0] and flag_01==0:
                                                #print(res1[-leng:]) #'ㅆ'
                                                #print(n[0])
                                                flag_01=1
                                                
                                                res3 = jam.jamo[i][:lis_i[i][j]-leng]
                                                print(res3)
                                                res4 = jam.jamo[i][lis_i[i][j]-leng:]
                                                print(res4)
                                                print(n[1])
                                                res4 = res4.replace(dic[k][0], n[1], 1)
                                                element = res3+res4
                                                print(element)
                                        if flag_01==0:
                                            res2 = res2.replace(dic[k][0], dic_e[m][2], 1)
                                            element = res1+res2
                    
            jam.jamo[i] = element
            #print(element)
        union(space_location, jam.jamo)
            #print(lis[i])
        
        #union(space_location, jam.jamo)
        return jam.make_one()
        
        
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        if result[-1] =='?' or result[-1] =='.' or result[-1] =='!' or result[-1] =='\"':
            result = result
        else:
            result = result+'.'
            flag = 1
        
        res = self.to_low(result)
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res

In [88]:
ch = Changer()

In [89]:
stc = '그건 제가 잘 모르겠습니다.'
tt = ch.processText(stc)
print(tt)

[1, 2, 3]
ㅁㅗㄹㅡㄱㅔㅆ
ㅅㅡㅂㄴㅣㄷㅏ
[['ㅆ', 'ㄷㅏ'], ['ㅎ', 'ㄷㅏ'], ['ㅄ', '다'], ['ㅂ', 'ㄷㅏ'], ['ㅌ', 'ㄷㅏ'], ['ㅍ', 'ㄷㅏ'], ['ㄵ', 'ㄷㅏ']]
ㅆ
ㅁㅗㄹㅡㄱㅔ
ㅆㅅㅡㅂㄴㅣㄷㅏ
ㄷㅏ
ㅁㅗㄹㅡㄱㅔㅆㄷㅏ
ㅆ
ㅆ
ㅆ
ㅆ
ㅆ
ㅆ
그건 제가 잘 모르겠다.


In [22]:
# for k in range(len(dic)):
#                         if isException(dic[k])==0:
#                             #print(jam.jamo[i][lis_i[i][j]:lis_i[i][j+1]])
#                             #print(dic[k][0])
#                             #print(dic[k][1])
                            
#                             #print('')
#                             if dic[k][0] in jam.jamo[i][lis_i[i][j]:lis_i[i][j+1]]:
#                                 #print('qq')
#                                 res1 = jam.jamo[i][:lis_i[i][j]]
#                                 res2 = jam.jamo[i][lis_i[i][j]:]
#                                 res2 = res2.replace(dic[k][0], dic[k][1], 1)
#                                 element = res1+res2
                                
#                         if isException(dic[k])==1:
                            
#                             if dic[k][0] in jam.jamo[i][lis_i[i][j]:lis_i[i][j+1]]:
                                
#                                 leng = len(EXC_deal_l[])

SyntaxError: invalid syntax (<ipython-input-22-1437a5de5480>, line 19)

In [95]:
lis_beta = ['EF','EP','EP+EF']

lis_end = [
    
    'ㅂㄴㅣㄷㅏ',
    'ㄴㅏㅇㅛ', 'ㅇㅡㄹㄲㅏㅇㅛ', 'ㅇㅣㄹㄲㅏㅇㅛ', 'ㅇㅡㄴㄱㅏㅇㅛ', 'ㅇㅣㄴㄱㅏㅇㅛ','ㅇㅛ',
    'ㅈㅛ',
    'ㅅㅣㅂㅅㅣㅇㅗ', 'ㅅㅣㅇㅗ', 'ㅇㅗ',
    'ㄴㅣㄲㅏ', 'ㄲㅏ', 'ㄹㄲㅏ',
    
]

In [112]:
def delete_end(w, t, list_tag, list_end): #list_tag
    
    lis_w = []
    lis_t = []
    number = len(w)
    print(len(w))
    print(len(t))
    for i in range(len(w)):
        ele = pre_target_b(t[i], lis_beta, 'EF')
        res1 = ''
        res2 = ''
        print(ele)
        if ele not in list_tag:
            res1 = w[i]
            print(res1)
            res2 = t[i]
            print(res2)
        elif ele=='B+EF':
            flag = 0
            for j in list_end:
                if j in w[i]:
                    flag=1
                    ind = w[i].index(j)
                    res1 = w[i][:ind]
                    res2 = t[i]
            if flag==0:
                res1 = w[i]
                res2 = t[i]
        lis_w.append(res1)
        lis_t.append(res2)
        number = number-1
    return lis_w, lis_t

def prepro_b(input):
    lis_w, lis_t = to2lists(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t(lis_t)
    union_w(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    jam3 = Jamodealer(lis_tag)
    
    number, jam1.jamo, jam3.jamo = delete_end(jam1.jamo, jam3.jamo, lis_beta, lis_end)
    
    
    for i in range(len(data_w)-number-1):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1
        
        #element = element[:-1]
        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
    
    return data_w, data_t, lis_ind

def pre_target_b(input, lis, target):#EF
    result = input
    if target in input:
        res = input.split('/')
        for j in range(len(res)):
            if target in res[j] and '+' in res[j]:
                loc = res[j].index('+')
                wd = res[j][:loc]
                print(wd)
                if wd not in lis:
                    res[j] = re.sub(wd, 'B', res[j])
        result = '/'.join(res)
    return result

In [118]:
input_3 = '정말 바보십니까? '

In [119]:
result = input_3

space_list = rememberSpace(input_3,' ')    
lis_w, lis_t, lis_i = prepro(result)
print(lis_w)
space_location = convertSpace(space_list, lis_w)
jam = Jamodealer(lis_w)

words, tags = to2lists(result)
jam_pre = Jamodealer(words)
print(tags)
lis_word, lis_tag,  = delete_end(jam_pre.jamo, tags, lis_beta, lis_end)
print(lis_word)

union(space_location, jam.jamo)

['정말', '바보십니까', '?']
['MAG', 'NNG', 'VCP+EP+EF', 'SF']
4
4
MAG
ㅈㅓㅇㅁㅏㄹ
MAG
NNG
ㅂㅏㅂㅗ
NNG
VCP
B+EP+EF
ㅅㅣㅂㄴㅣㄲㅏ
VCP+EP+EF
SF
?
SF
['ㅈㅓㅇㅁㅏㄹ', 'ㅂㅏㅂㅗ', 'ㅅㅣㅂㄴㅣㄲㅏ', '?']
